In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, lit, avg, when, broadcast

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
gender_df = spark.table("database.gender")
popu_df = spark.table("database.population")

gender_df = gender_df.filter(col("nome_indicador").like("%Unemployment with advanced%"))
gender_df = gender_df.drop(col("cod_pais"), col("cod_indicador"))
gender_df = gender_df.withColumnRenamed("nome_pais", "pais_name")
gender_df = gender_df.withColumnRenamed("nome_indicador", "indicador_name")

popu_df = popu_df.filter(col("cod_indicador").like("%SP.POP.TOTL%"))
popu_df = popu_df.drop(col("cod_pais"), col("cod_indicador"), col("nome_indicador"), col("Country_name"), col("Country_code"), col("Indicator_name"), col("Indicator_code"))
popu_df = popu_df.withColumnRenamed("valor", "valorI")
popu_df = popu_df.withColumnRenamed("nome_pais", "pais_name")
popu_df = popu_df.withColumn("ano", col("ano").cast(IntegerType()))

In [3]:
paises = ["%Liechtenstein%", "%Germany%", "%Denmark%", "%Switzerland%", "%Poland%", "%Czech%", 
          "%Slovakia%", "%Slovenia%", "%Hungary%", "%Austria%"]
anos = ["2010", "2011", "2012", "2013", "2014", "2015"]

gender_df = gender_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3])
).filter(
    col("ano").isin(anos) &
    col("valor").isNotNull()
)

popu_df = popu_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3])
).filter(
    col("ano").isin(anos) &
    col("valorI").isNotNull()
)

In [4]:
literacia_df = gender_df.join(popu_df, ["pais_name", "ano"], "left")
literacia_df = literacia_df.withColumn("valorI", ((col("valor") / 100) * (col("valorI") / 2)).cast("int"))
literacia_df = literacia_df.withColumnRenamed("pais_name", "paises")

In [5]:
literacia_df = literacia_df.withColumn("genero",
    when(col("indicador_name").like("% female%"), "F")
    .when(col("indicador_name").like("% male%"), "M")
)

literacia_df = literacia_df.filter(col("genero").isNotNull())

literacia_df = literacia_df.drop(col("indicador_name"))

In [6]:
literacia_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaEnrolmentT/")

In [7]:
spark.sql(
    """
    SELECT * FROM database.EnrolmentTGold
    """
).toPandas()

,paises,ano,valor,valorI,genero
0,Denmark,2010,4.46,123713,F
1,Denmark,2011,5.39,150126,F
2,Denmark,2012,5.27,147337,F
3,Denmark,2013,4.76,133635,F
4,Denmark,2014,5.04,142215,F
5,Denmark,2015,4.99,141802,F
6,Denmark,2010,4.96,137582,M
7,Denmark,2011,4.68,130351,M
8,Denmark,2012,4.47,124971,M
9,Denmark,2013,4.57,128301,M


In [8]:
spark.stop()